In [65]:
from konlpy.tag import Komoran
import pickle
import re
import pandas as pd

In [66]:
#사용자 단어사전 구축 전
komoran = Komoran()
text = "6개월간 올라온 블랙핑크 영상 찾아줘."
pos = komoran.pos(text)

print(pos)

[('6', 'SN'), ('개월', 'NNB'), ('간', 'NNB'), ('올라오', 'VV'), ('ㄴ', 'ETM'), ('블랙', 'NNP'), ('핑크', 'NNP'), ('영상', 'NNP'), ('찾', 'VV'), ('아', 'EC'), ('주', 'VX'), ('어', 'EF'), ('.', 'SF')]


## 위의 결과에서 
## 1. '6개월'을 기간 키워드로
## 2. '블랙' 과 '핑크'로 분리된 결과를 '블랙핑크' 한 단어 키워드로
## 3. NNP(대명사)로 분류된 키워드 중 '블랙핑크'만 주제 키워드로 가져와야 한다. 

In [72]:

class Preprocess_text:
	# 생성자
	def __init__(self, word2index_dic="", userdic=""): 
		# 단어 인덱스 사전 불러오기
		if word2index_dic != "":
			f = open(word2index_dic, "rb")
			self.word_index = pickle.load(f)
			f.close()
		else:
			self.word_index = None

		# 형태소 분석기 초기화
		self.komoran = Komoran(userdic=userdic)
        

		# 제외할 품사
		# 참조: https://docs.komoran.kr/firststep/postypes.html
		self.exclusion_tags = [
			"JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ", "JX", "JC", # 관계언 제거
			"SF", "SP", "SS", "SE", "SO", # 기호 제거
			"EP", "EF", "EC", "ETN", "ETM", # 어미 제거
			"XSN", "XSV", "XSA", # 접미사 제거 
		]
        
        
        

	# 형태소 분석기 POS tagger (래퍼 함수)
	def pos(self, sentence):
		return self.komoran.pos(sentence)

	# 불용어 제거 후 필요한 품사 정보만 가져오기
	def get_keywords(self, pos, without_tag=False):
		f = lambda x: x in self.exclusion_tags
		word_list = []
		for p in pos:
			if f(p[1]) is False: # 불용어 리스트에 없는 경우에만 저장
				word_list.append(p if without_tag is False else p[0])
		return word_list


def get_duration_keywords(self, pos):
    #기간 키워드 추출 결과 저장 리스트 초기화
    #duration_keywords=[]
    keyword_pattern = r'(\d+년|\d+개월|최근)'
    
    #duration_keywords = [word for word, pos_tag in pos if pos_tag == 'NNBC']
    for word,pos_tag in pos:
            duration_keyword = re.findall(keyword_pattern, word)
            if duration_keyword:
                #duration_keywords.append(duration_keyword[0])
                duration_keywords=duration_keyword[0]
                
    
    print("기간 키워드:", duration_keywords)
    return duration_keywords

#주제 키워드 추출 메소드 작성
def get_title_keywords(self, pos_tag,stopdic='',duration_keywords=''):
    nnp_keywords=[]
    stop_keywords=[]
    duration_keywords=duration_keywords
    for word_tag in pos_tag:
        if word_tag[1] == 'NNP':
            nnp_keywords.append(word_tag[0])
    #고유명사 키워드 리스트에서 기간 키워드 제외
    if duration_keywords in nnp_keywords:
        nnp_keywords.remove(duration_keywords)
    print("추출된 고유명사:", nnp_keywords)
    
    #불용어 사전 불러오기
    if stopdic!="":
        stopwords_path = stopdic
        with open(stopwords_path, 'r', encoding='utf-8') as file:
            #BOM문자,개행문자 제거
            stop_keywords =[line.strip('\ufeff') for line in file.readlines()]
            stop_keywords =[l.strip() for l in stop_keywords]

    print("불용어 키워드:", stop_keywords)
    #고유명사 키워드에서 불용어 키워드 제외_주제 키워드만 남김
    for stop_word in stop_keywords:
        if stop_word in nnp_keywords:
            nnp_keywords.remove(stop_word)
            #print("제거된 불용어 키워드:", stop_word)
     
    print("추출된 주제 키워드:", nnp_keywords)
    return nnp_keywords
    
# Preprocess 클래스에 메서드를 추가
Preprocess_text.get_duration_keywords = get_duration_keywords
Preprocess_text.get_title_keywords =get_title_keywords
    
    

In [4]:
#정규표현식으로 기간 키워드 찾기
sentence = "6개월간 올라온 블랙핑크 영상 찾아줘"
sentence2 = "1년간 올라온 블랙핑크 영상 찾아줘"
sentence3 = "최근 올라온 블랙핑크 영상 찾아줘"


# 정규 표현식을 사용하여 기간 키워드를 추출합니다.
keyword_pattern = r'(\d+개월|\d+년|최근)'

# 정규 표현식과 매칭되는 부분을 찾습니다.
matches = re.findall(keyword_pattern, sentence)
matches = re.findall(keyword_pattern, sentence2)
matches = re.findall(keyword_pattern, sentence3)

# 매칭된 결과를 출력합니다.
print(matches)
#print(matches2)
#print(matches3)


['최근']


In [14]:
#불용어사전
stopwords_path = 'stopwords.txt'
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stopwords = file.read()
    
# 불용어 사전 내용 출력
print(stopwords)

﻿영상
데이터


In [6]:
# 파일 경로 설정
#file_path = 'C:\\Users\\user\\Desktop\\nlpToDf\\user_dic_test.txt'
file_path = 'C:\\Users\\user\\Desktop\\nlpToDf\\user_dic.tsv'
# 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# 읽어온 내용 출력
print(content)



﻿블핑	NNP
블랙핑크	NNP
블랙 핑크	NNP
6개월	NNP
blackpink	NNP


In [73]:

#sent = "최근 올라온 블랙핑크 영상 찾아줘"
sent = "6개월간 올라온 블랙핑크 영상 찾아줘"

# 전처리 객체 생성
#p = Preprocess(userdic='../utils/user_dic.tsv')
#komoran.setUserDic("user_dic.tsv");
p = Preprocess_text(userdic='user_dic.tsv')
#p = Preprocess()

# 형태소 분석기 실행
pos = p.pos(sent)

# 품사 태그와 같이 키워드 출력
ret = p.get_keywords(pos, without_tag=False)
print("형태소 추출/품사태깅:",ret)

# 품사 태그 없이 키워드 출력
ret = p.get_keywords(pos, without_tag=True)
print("형태소 추출:",ret)

duration_keywords = p.get_duration_keywords(pos)

title_keywords = p.get_title_keywords(pos,stopdic='stopwords.txt',duration_keywords=duration_keywords)

#추출한 키워드 df생성
for keyword in title_keywords:
    title_keywords_all=' '.join(title_keywords)

df = pd.DataFrame({'period': [duration_keywords], \
        'title': title_keywords_all})
print(df)

형태소 추출/품사태깅: [('6개월', 'NNP'), ('간', 'NNB'), ('올라오', 'VV'), ('블랙핑크', 'NNP'), ('영상', 'NNP'), ('찾', 'VV'), ('주', 'VX')]
형태소 추출: ['6개월', '간', '올라오', '블랙핑크', '영상', '찾', '주']
기간 키워드: 6개월
추출된 고유명사: ['블랙핑크', '영상']
불용어 키워드: ['영상', '데이터']
추출된 주제 키워드: ['블랙핑크']
  period title
0    6개월  블랙핑크


In [69]:
sent2 = "최근 올라온 blackpink 데이터 찾아줘"

# 전처리 객체 생성
#p = Preprocess(userdic='../utils/user_dic.tsv')
p = Preprocess_text(userdic='user_dic.tsv')

# 형태소 분석기 실행
pos_tag = p.pos(sent2)

# 품사 태그와 같이 키워드 출력
ret = p.get_keywords(pos_tag, without_tag=False)
print("형태소 추출/품사태깅:",ret)

# 품사 태그 없이 키워드 출력
ret = p.get_keywords(pos_tag, without_tag=True)
print("형태소 추출:",ret)

#기간 키워드 출력
duration_keywords = p.get_duration_keywords(pos_tag)
#print("기간 키워드2:", duration_keywords)

nnp_keywords = p.get_title_keywords(pos_tag)



형태소 추출/품사태깅: [('최근', 'NNG'), ('올라오', 'VV'), ('blackpink', 'NNP'), ('데이터', 'NNG'), ('찾', 'VV'), ('주', 'VX')]
형태소 추출: ['최근', '올라오', 'blackpink', '데이터', '찾', '주']
기간 키워드: 최근
추출된 고유명사: ['blackpink']
불용어 키워드: []
추출된 주제 키워드: ['blackpink']


In [9]:
#기간 키워드와 주제 키워드를 df로 저장


df1 = pd.DataFrame({'period': [duration_keywords], \
        'title': nnp_keywords})
print(df1)

  period      title
0     최근  blackpink
